# Modifications de résultats des sondages

## Import préliminaires des librairies

In [1]:
import re
import numpy as np
import pandas as pd
import locale 
from locale import atof
# Régression linéaire
from sklearn.linear_model import LinearRegression

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

# Perso
from importlib.machinery import SourceFileLoader
tools = SourceFileLoader('methods', '../methods.py').load_module()
from methods import *

In [83]:
fp="rapport/2020-06-03.pdf"
text = extract_text(fp,page_numbers=[16])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
# print(text.find(question1))
# print(text.find(question2))
# print(len(question2))
print(re.sub('\.\.\. (?![\n])','...\n',text))

Le classement des personnalités
Auprès des sympathisants de la gauche

Question : Pour chacune des personnalités suivantes, indiquez si vous en avez une image positive ou une image négative ?
En %

45

20

14

Nicolas Hulot
Olivier Besancenot
Christiane Taubira
Jean-Luc Mélenchon
Anne Hidalgo
Benoît Hamon
François Ruffin
Ségolène Royal
Yannick Jadot
François Hollande
Jean-Yves Le Drian
Arnaud Montebourg
Bernard Cazeneuve
François Bayrou
Olivier Véran
Olivier Faure
Bruno Le Maire
Xavier Bertrand
François Baroin
Marlène Schiappa
Jean-Michel Blanquer
Rachida Dati
Marion Maréchal
Nicolas Sarkozy
Valérie Pécresse
Manuel Valls
Marine Le Pen
Christophe Castaner
Gérald Darmanin
Gérard Larcher
Nicolas Dupont-Aignan
Sibeth Ndiaye
Muriel Pénicaud
Christian Jacob
Laurent Wauquiez

17

22

18

7

14

18

7
11

6
8

5

10

4
5
3

6

3
3
3
2
4
4
4
2
4
4
2
3
2
1

4

14
14
14
16
13
13
15
13

12
13
7

36
30

34

26

29

23

27
24
26
21

24
22
20
21

44

31

33

43

34
28

12

18

25

33

22

31

19
22



In [85]:
with open(fp,"rb") as pdf_file:
    pdf_reader= PyPDF2.PdfFileReader(pdf_file)
    isEncrypted = pdf_reader.getIsEncrypted()
    nbPages= pdf_reader.numPages

    # Define columns
    page_content=pdf_reader.getPage(15).extractText()
print(page_content)

10
9
6
5
5
5
5
8
7
6
4
8
4
4
3
3
3
6
3
4
3
2
6
3
2
3
4
3
3
2
2
2
2
1
2
35
26
26
27
27
25
24
20
21
22
24
18
22
22
22
21
21
17
19
17
18
19
14
17
18
16
14
15
14
15
11
11
11
11
6
21
21
21
19
19
16
18
17
18
16
22
19
20
22
28
23
21
21
28
22
18
23
17
29
26
21
16
22
23
21
18
20
27
19
16
16
30
20
17
21
18
16
43
35
29
28
34
24
29
22
22
24
42
35
30
25
38
22
33
20
36
18
29
18
33
21
32
42
19
48
18
14
27
32
28
36
37
12
19
27
22
21
30
23
25
31
31
14
15
27
36
18
41
18
34
24
48
31
42
29
48
35
18
50
28
Nicolas Hulot
Nicolas Sarkozy
Olivier Véran
Xavier Bertrand
Bruno Le Maire
Jean-Yves Le Drian
François Baroin
Marine Le Pen
Marion Maréchal
Olivier Besancenot
Anne Hidalgo
Christiane Taubira
Valérie Pécresse
Rachida Dati
François Bayrou
Bernard Cazeneuve
Jean-Michel Blanquer
Jean-Luc Mélenchon
François Hollande
Benoît Hamon
Gérald Darmanin
Christophe Castaner
François Ruffin
Ségolène Royal
Arnaud Montebourg
Nicolas Dupont-Aignan
Yannick Jadot
Laurent Wauquiez
Gérard Larcher
Marlène Schiappa
Christian Jaco

# Extraction des données avec PyPDF2
PyPDF2 semble être constant dans la forme d'extraction des données de Elab.
La structure des données ainsi que les détails d'extraction sont précisés ci-dessous:
* Elabe fourni les données détaillées pour deux personnalités que sont le président de la République et le premier ministre. Les noms n'étant pas indiqués sur la page de données, il faut les extraire ailleurs ou les préciser à la main.
* Il y a deux pages successives de données par individus
* Les catégories principales sont indiquées en capitales
* Il existe des catégories secondaires et tertiaires sans différenciation possible. Elles seront traités comme sous catégories.
* La base de chaque catégories n'est pas précisée, il faudra donc utiliser de la régression linéaire pour tenter de les retrouver
* Les trous dans les données ne sont pas visible. Il faut un système de vérification.
* Les variations ne sont pas données

__Format des données__
* Les données sont fournis en lisant les lignes dans l'ordre
* Chaque cellule est extraite par PyPDF2 une à la fois en introduisant un saut de ligne
* Les blancs dans les données ne créént pas de fusion d'information mais disparaissent sans faire de double saut de lignes
* Le caractère '__-__' compromet les cellules et induit un ou plusieurs sauts malencontreux :
    * Si le '-' apparait au milieu d'un texte, il induit un _double saut de ligne_ (ex: 18-24 ans)
    * Si le '-' apparait à la fin d'un texte il induit un _seul_ saut de ligne supplémentaire avant sont apparition (CSP-)  
* Il faudra donc préformater le texte au préalable


## Extract content (to delete later)

In [4]:
def get_page_content(num_page,fp)->str:
    with open(fp,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages
        return pdf_reader.getPage(num_page-1).extractText()

    # Define columns
page_content=get_page_content(2,fp)
page_content

"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de \n1 003  \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n2 et 3 mars 2021\n.\n04/03/2021\nL'Observatoire politique \n\nMars 2021\n"

## Préformatage

In [5]:
##Préformatage
def preformatage(page_content)->str:
    #Remplacement des données d'age
    page_content=page_content.replace('18\n-\n24 ans','18-24 ans')
    page_content=page_content.replace('25\n-\n34 ans','25-34 ans')
    page_content=page_content.replace('35\n-\n49 ans','35-49 ans')
    page_content=page_content.replace('50\n-\n64 ans','50-64 ans')
    # Remplacement CSP-
    page_content=page_content.replace('CSP\n-','CSP-')
    # Remplacement EELV
    page_content=page_content.replace('Europe Ecologie \n-\nLes Verts','Europe Ecologie Les Verts')
    # Noms de candidats
    page_content=page_content.replace('Jean\n-\n','Jean-')
    # Remplacement /
    page_content=page_content.replace('/ \n','/ ')
    page_content=page_content.replace('\n/','/')
    # Remplacement Total .. en minuscule
    page_content=page_content.replace('TOTAL GAUCHE','Total Gauche')
    page_content=page_content.replace('TOTAL EN MARCHE / MODEM','Total en Marche / MODEM')
    page_content=page_content.replace('TOTAL DROITE','Total Droite')
    # Suppression fin de texte
    page_content=page_content.replace('\nLa cote de confiance du Président de la République','')
    page_content=page_content.replace('\nLa cote de confiance du Premier ministre','')
    # Suppression des notes de bas de pages
    page_content=page_content.replace(' (*)','')
    page_content=page_content.replace('(*)','')
    # Suppression du dernier saut de ligne
    page_content=page_content.rstrip('\n')
    return page_content

preformatage(page_content)


"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de \n1 003  \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n2 et 3 mars 2021\n.\n04/03/2021\nL'Observatoire politique \n\nMars 2021"

## Extraction de la base

In [6]:
def extract_base(fp,num_page=2)->int:
    page_content=get_page_content(num_page,fp)
    text_deb='Echantillon de \n'
    text_fin='\npersonnes'
    i_deb=page_content.find(text_deb)+len(text_deb)
    i_fin=page_content.find(text_fin)
    base=int(page_content[i_deb:i_fin].rstrip().replace(' ',''))
    return base
extract_base(fp)

1003

## Création du modèle de Dataframe

In [7]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])

## Extraction de page formattée

In [8]:
def extract_series(data,i,columns,nom,base,date,prv_categorie)->pd.Series:
    """Extrait une serie de data à partir de l'indice i jusqu'à la prochaine valeur non digit

    Args:
        data (list): Toutes les données dans une liste 
        i (int): Indice courant dans les données
        columns (list): Liste des noms de colonnes 
        nom (str): Nom de la personnalité
        base (int): Nombre de sondés
        date (datetime.date): Date du sondage

    Returns:
        pd.Series: La serie contenant les valeurs à stocker
        int: new index
    """
    values=[nom,base,date]
    #Catégorie
    if data[i].isupper() and '+' not in data[i] and '-' not in data[i]:
        values.append(data[i])
        i+=1
    else:
        values.append(prv_categorie)
    #Groupe
    values.append(data[i])
    # On saut la cellule qui contient le 100% et qui ne sert a rien
    i+=2
    while i<len(data) and re.match('^\d+$',data[i]):
        values.append(float(data[i]))
        i+=1
    # Vérification
    while len(values)!=len(columns):
        print(values)
        j=int(input('Où faut-il ajouter un zéro ? (1-7)'))+4
        values=values[:j]+[0.0]+values[j:]
    return pd.Series(values,index=columns),i



def extract_detailled_page(page_content,df,nom,base,date)->pd.DataFrame:
    """
    Extrait une page du pdf et l'ajoute au dataframe.
    
    Parameters
    ----------
    page_content: texte formatté de la page dot l'on souhaite extraire les données
    df: Dataframe pour stocker les données
    nom: Nom de la personnalité 
    base: Base d'étude (à compléter par regression plus tard)
    date: Date du sondage

    Returns
    -------
    Dataframe
        Dataframe df avec les nouvelles données
    """
    indice_debut=page_content.find('Ensemble')
    data=page_content[indice_debut:].split('\n')
    i=0
    cat='Ensemble'
    while i < len(data):
        if not re.match('^\d+$',data[i]):
            s,i=extract_series(data,i,df.columns,nom,base,date,cat)
            cat=s.Categorie
            df=df.append(s,ignore_index=True)
    return df


## Extraction pour une personnalité

In [9]:
def extract_detailled_perso(df,fp,page_indexes,nom,base,date)->pd.DataFrame:
    """Extrait les données sur toutes les pages pour une personnalité

    Args:
        df (pandas.Dataframe): Dataframe stockant les données
        fp (str): filepath du fichier à lire
        page_indexes (list): Liste des pages à extraire
        nom (str): Nom de la personnalité
        base (int): Nb de sondés
        date (datetime.date): Date du sondate
        inline (bool): Si True modification en ligne

    Returns:
        pd.DataFrame: Dataframe étendu des nouvelles données
    """
    for i in page_indexes:
        page_content=preformatage(get_page_content(i,fp))
        df=extract_detailled_page(page_content,df,nom,base,date)
    df=df.drop_duplicates(ignore_index=True)
    return df

        

In [34]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])
date='2020-06-03'
filepath='rapport/{}.pdf'.format(date)
president='Emmanuel MACRON'
page_president=[10,11]
ministe='Edouard PHILIPPE'
page_ministre=[13,14]
base=extract_base(filepath)
df=extract_detailled_perso(df,filepath,page_president,president,base,date)
df=extract_detailled_perso(df,filepath,page_ministre,ministe,base,date)
df

['Emmanuel MACRON', 1002, '2020-06-03', 'PREFERENCE PARTISANE', 'La République en Marche !', 31.0, 65.0, 4.0, 1.0, 96.0, 4.0]
['Emmanuel MACRON', 1002, '2020-06-03', 'PREFERENCE PARTISANE', 'Les Républicains', 6.0, 36.0, 35.0, 23.0, 42.0, 58.0]
['Emmanuel MACRON', 1002, '2020-06-03', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'Benoît HAMON', 4.0, 30.0, 32.0, 34.0, 34.0, 66.0]
['Emmanuel MACRON', 1002, '2020-06-03', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'François FILLON', 10.0, 36.0, 25.0, 29.0, 46.0, 54.0]
['Edouard PHILIPPE', 1002, '2020-06-03', 'PROFESSION DU REPONDANT', 'Cadre, profession intellectuelle supérieure', 13.0, 39.0, 26.0, 22.0, 52.0, 48.0]
['Edouard PHILIPPE', 1002, '2020-06-03', 'PREFERENCE PARTISANE', 'Le Rassemblement National', 14.0, 20.0, 62.0, 3.0, 14.0, 83.0]


,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif
0,Emmanuel MACRON,1002,2020-06-03,Ensemble,Ensemble,7.0,26.0,28.0,34.0,5.0,33.0,62.0
1,Emmanuel MACRON,1002,2020-06-03,SEXE DU REPONDANT,Homme,8.0,28.0,25.0,34.0,5.0,36.0,59.0
2,Emmanuel MACRON,1002,2020-06-03,SEXE DU REPONDANT,Femme,6.0,23.0,31.0,34.0,6.0,29.0,65.0
3,Emmanuel MACRON,1002,2020-06-03,AGE DU REPONDANT,18-24 ans,11.0,24.0,31.0,22.0,12.0,35.0,53.0
4,Emmanuel MACRON,1002,2020-06-03,AGE DU REPONDANT,25-34 ans,11.0,30.0,22.0,32.0,5.0,41.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (1ER TOUR),Marine LE PEN,1.0,17.0,25.0,54.0,3.0,18.0,79.0
84,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (1ER TOUR),"Abstention, vote blanc ou nul",5.0,23.0,35.0,27.0,10.0,28.0,62.0
85,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Emmanuel MACRON,18.0,45.0,24.0,9.0,4.0,63.0,33.0
86,Edouard PHILIPPE,1002,2020-06-03,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Marine LE PEN,2.0,16.0,24.0,55.0,3.0,18.0,79.0


## Correction des bases

In [35]:
for c  in df.Categorie.unique():
    print('{} :[{}]'.format(c,','.join(["'"+x+"'" for x in df[df.Categorie==c].Groupe.unique()])))

Ensemble :['Ensemble']
SEXE DU REPONDANT :['Homme','Femme']
AGE DU REPONDANT :['18-24 ans','25-34 ans','35-49 ans','50-64 ans','65 ans et plus']
PROFESSION DU REPONDANT :['Actifs','Artisan, commerçant','CSP+','Cadre, profession intellectuelle supérieure','Profession intermédiaire','CSP-','Employé','Ouvrier','Inactifs','Retraité','Autre Inactif']
CATEGORIE D'AGGLOMERATION :['Communes rurales','Communes urbaines de province','De 2.000 à moins de 20.000 hab.','De 20.000 à moins de 100.000 hab.','100.000 hab. et plus','Agglomération parisienne']
PREFERENCE PARTISANE :['Total Gauche','La France Insoumise','Europe Ecologie Les Verts','Parti Socialiste / Génération.s/ PRG','Total en Marche / MODEM','La République en Marche !','Total Droite','Les Républicains','Le Rassemblement National','Sans préférence partisane']
VOTE PRESIDENTIELLE 2017 (1ER TOUR) :['Jean-Luc MELENCHON','Benoît HAMON','Emmanuel MACRON','François FILLON','Marine LE PEN','Abstention, vote blanc ou nul']
VOTE PRESIDENTIELLE 2

In [36]:
# CategorieX, GroupeX, CategorieY, GroupeY
relations=[
    ('SEXE DU REPONDANT' ,['Homme','Femme'],'Ensemble','Ensemble'),
    ('AGE DU REPONDANT' ,['18-24 ans','25-34 ans','35-49 ans','50-64 ans','65 ans et plus'],'Ensemble','Ensemble'),
    ('PROFESSION DU REPONDANT' ,['Actifs','Inactifs'],'Ensemble','Ensemble'),
    ('PROFESSION DU REPONDANT' ,['Artisan, commerçant','CSP+','CSP-'],'PROFESSION DU REPONDANT','Actifs'),
    ('PROFESSION DU REPONDANT' ,['Cadre, profession intellectuelle supérieure','Profession intermédiaire'],'PROFESSION DU REPONDANT','CSP+'),
    ('PROFESSION DU REPONDANT' ,['Employé','Ouvrier'],'PROFESSION DU REPONDANT','CSP-'),
    ('PROFESSION DU REPONDANT' ,['Retraité','Autre Inactif'],'PROFESSION DU REPONDANT','Inactifs'),
    ("CATEGORIE D'AGGLOMERATION" ,['Communes rurales','Communes urbaines de province','Agglomération parisienne'],'Ensemble','Ensemble'),
    ("CATEGORIE D'AGGLOMERATION" ,['De 2.000 à moins de 20.000 hab.','De 20.000 à moins de 100.000 hab.','100.000 hab. et plus'],"CATEGORIE D'AGGLOMERATION",'Communes urbaines de province'),
    ('PREFERENCE PARTISANE' ,['Total Gauche','Total en Marche / MODEM','Total Droite','Le Rassemblement National','Sans préférence partisane'],'Ensemble','Ensemble'),
    ('PREFERENCE PARTISANE' ,['La France Insoumise','Europe Ecologie Les Verts','Parti Socialiste / Génération.s/ PRG'],'PREFERENCE PARTISANE','Total Gauche'),
    ('PREFERENCE PARTISANE' ,['La République en Marche !'],'PREFERENCE PARTISANE','Total en Marche / MODEM'),
    ('PREFERENCE PARTISANE' ,['Les Républicains'],'PREFERENCE PARTISANE','Total Droite'),
    ('VOTE PRESIDENTIELLE 2017 (1ER TOUR)' ,['Jean-Luc MELENCHON','Benoît HAMON','Emmanuel MACRON','François FILLON','Marine LE PEN','Abstention, vote blanc ou nul'],'Ensemble','Ensemble'),
    ('VOTE PRESIDENTIELLE 2017 (2ND TOUR)' ,['Emmanuel MACRON','Marine LE PEN','Abstention, vote blanc ou nul'],'Ensemble','Ensemble'),   
]

In [37]:
df=recompute_base(df,relations,df.columns[5:])

In [38]:
df.head()
# pd.to_pickle(df,'data/{}.p'.format(date))

# Resultats de toutes les personnalités
## Extraction des valeurs

In [92]:

def preformatage_personnalite(page_content)->str:
    page_content=page_content.replace('\n-\n','\n-')
    return page_content

def first_index_not_numerical(list_content)->int:
    """Retourne l'index de la première valeur qui ne soit pas une chaine de caractère représentant un nombre

    Args:
        list_content (List[str]): Liste du contenant une ligne du texte par cellule

    Returns:
        int: index de la position recherchée. Retourne -1 si non trouvée
    """
    for i in range(len(list_content)):
        if not re.match('^\d+$',list_content[i]):
            return i
    return -1
    

def extract_detailled_page_all_personnalite(page_content,columns,base,date)->pd.DataFrame:
    # Données fournies dès le début de la page
    data=page_content.split('\n')
    index_names=first_index_not_numerical(data)
    # Création des données pour faire le dataframe
    values=np.array([int(x) for x in data[:index_names]]).reshape(5,-1).T
    nb_perso=values.shape[0]

    total_pos=values[:,0:2].sum(axis=1).reshape(-1,1)
    total_neg=values[:,2:4].sum(axis=1).reshape(-1,1)

    noms=data[index_names:index_names+nb_perso]
    noms=['{} {}'.format(x.split(' ')[0],x.split(' ')[1].upper()) for x in noms]
    noms=np.array(noms).reshape(-1,1)
    

    variations=np.array(data[-1-nb_perso:-1])
    variations[variations=='=']=0
    variations=variations.reshape(-1,1)

    base_np=np.array([[base]]*nb_perso)

    date_np=np.array([[date]]*nb_perso)

    cat=np.array([['Ensemble']]*nb_perso)

    groupe=np.array([['Ensemble']]*nb_perso)

    # Création du tableau numpy
    tab=np.append(noms,base_np,axis=1)
    tab=np.append(tab,date_np,axis=1)
    tab=np.append(tab,cat,axis=1)
    tab=np.append(tab,groupe,axis=1)
    tab=np.append(tab,values,axis=1)
    tab=np.append(tab,total_pos,axis=1)
    tab=np.append(tab,total_neg,axis=1)
    tab=np.append(tab,variations,axis=1)
    df=pd.DataFrame(tab,columns=columns)
    return df

def extract_personnalite(fp,num_page,columns):
    page_content=preformatage_personnalite(get_page_content(num_page,fp))
    df=extract_detailled_page_all_personnalite(page_content,columns,base,date)
    return df



In [106]:
COLUMNS_PERSO=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif','evolution_positif']
DTYPES_PERSO={'Nom':str,'Base':float,'Date':'datetime64[ns]','Categorie':str,'Groupe':str,'tres_positif':float,'positif':float,'negatif':float,'tres_negatif':float,'nsp':float,'total_positif':float,'total_negatif':float,'evolution_positif':float}

date='2020-06-03'
filepath='rapport/{}.pdf'.format(date)
base=extract_base(filepath)
df_perso=extract_personnalite(filepath,16,COLUMNS_PERSO)
df_perso=df_perso.astype(DTYPES_PERSO)
df_perso.dtypes

# pd.to_pickle(df_perso,'data_perso/{}.p'.format(date))

In [27]:

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

False


,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif


## Find mistakes and try to correct them

In [28]:
def find_fav_mistake(df):
    tfav=df['Très favorable']
    pfav=df['Plutôt favorable']
    stfav=df['ST Favorable']
    fav_df= df[np.abs(tfav+pfav-stfav)>0.11]
    return fav_df

def find_def_mistake(df):
    tdef=df['Très défavorable']
    pdef=df['Plutôt défavorable']
    stdef=df['ST Défavorable']
    def_df= df[np.abs(tdef+pdef-stdef)>0.11]
    return def_df

def find_nsp_mistake(df):
    stfav=df['ST Favorable']
    stdef=df['ST Défavorable']
    nsp=df['Nsp']
    tot_df= df[np.abs(stfav+stdef+nsp-100)>0.11]
    return tot_df

def find_mistake(df):
    fav_df= find_fav_mistake(df)
    def_df= find_def_mistake(df)
    tot_df=find_nsp_mistake(df)
    error_df=pd.concat([fav_df,def_df,tot_df])
    return error_df

In [29]:
def realign_fav(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très favorable","Plutôt favorable","ST Favorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_fav(df,i+1)
    return df

def realign_def(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très défavorable","Plutôt défavorable","ST Défavorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_def(df,i+1)
    return df

def realign_nsp(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["ST Favorable","ST Défavorable","Nsp"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_nsp(df,i+1)
    return df

    



In [30]:
def correct_mistakes(df):
    df_err=find_fav_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_fav(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_def_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_def(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_nsp_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_nsp(df_err[cond])
    df.loc[df_err.index]=df_err
    return df

In [31]:
df_err=find_mistake(df)
if len(df_err)>0:
    index=df_err.iloc[0:1].index[0]
    val=df_err.iloc[0].values[3:]
    print(val)
    correct=[]
    cols=["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    while len(correct)!=10:
        correct=input()
        correct=[float(x) for x in correct.split(' ')]
    print(correct)
    df.loc[index,cols]=correct

KeyError: 'Très favorable'

In [32]:
df_err=find_mistake(df)
df_err

KeyError: 'Très favorable'

In [ ]:

# df.to_pickle('data/'+filename+'.p')